In [1]:
import nltk

nltk.download('punkt')
import pandas as pd

[nltk_data] Downloading package punkt to /Users/niruksorp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os

df = pd.read_csv(os.path.join('data.csv'), names=['label', 'Title', 'Description'])
df['text'] = (df['Title'] + '. ' + df['Description'])
df.drop(columns=['Title', 'Description'], axis=1, inplace=True)
print(df.head())
print(df['text'][1])

   label                                               text
0      3  Carlyle Looks Toward Commercial Aerospace (Reu...
1      3  Iraq Halts Oil Exports from Main Southern Pipe...
2      3  Oil prices soar to all-time record, posing new...
3      3  Stocks End Up, But Near Year Lows (Reuters). R...
4      3  Money Funds Fell in Latest Week (AP). AP - Ass...
Iraq Halts Oil Exports from Main Southern Pipeline (Reuters). Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.


In [63]:
df['label']

,label,text
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...
1,3,Iraq Halts Oil Exports from Main Southern Pipe...
2,3,"Oil prices soar to all-time record, posing new..."
3,3,"Stocks End Up, But Near Year Lows (Reuters). R..."
4,3,Money Funds Fell in Latest Week (AP). AP - Ass...
...,...,...
119993,1,Pakistan's Musharraf Says Won't Quit as Army C...
119994,2,Renteria signing a top-shelf deal. Red Sox gen...
119995,2,Saban not going to Dolphins yet. The Miami Dol...
119996,2,Today's NFL games. PITTSBURGH at NY GIANTS Tim...


In [174]:
import re

def split_to_sent(text):
    sentences = re.split(
        r"(((?<!\w\.\w.)(?<!\s\w\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s(?=[A-Z]))|((?<![\,\-\:])\n(?=[A-Z]|\" )))", text)[
                ::4]
    return sentences

In [175]:
def split_to_words(sentence):
    words = re.findall(r"\w+@\w+\.\w+|\+\d{1,3}-\d{3}-\d{3}-\d{2}-\d{2}|\w+", sentence)
    return words

In [192]:
from nltk import WordNetLemmatizer
from nltk import SnowballStemmer
import nltk
nltk.download('wordnet')


def process_file():
    wnl = WordNetLemmatizer()
    sst = SnowballStemmer("english")
    counter = 0
    for index, row in df.iterrows():
        sentences = split_to_sent(row['text'])
        words_dic = []
        counter += 1
        for s in sentences:
            words_dic += split_to_words(s)
            words_dic.append("\n")
        lemmatized = []
        stemmed = []
        original = []
        for w in words_dic:
            w_processed = re.sub(r"[.!?,]$", "", w).lower() # убрать удаление точек и etc. отделить от токена и сохранить отдлеьно (1 балл доделать)
            lemmatized.append(wnl.lemmatize(w_processed))
            stemmed.append(sst.stem(w_processed))
            original.append(w_processed)
        save_to_file(original, lemmatized, stemmed, os.path.join(".", "assets", "annotated-corpus", "train", str(row['label']), str(counter) + ".tsv"))

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niruksorp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [190]:
def save_to_file(original, lemmatized, stemmed, id):
    with open(id, "w") as f:
        for i in range(len(original)):
            if original[i] == "\n":
                 print("", file=f)
            else:
                print(original[i], stemmed[i], lemmatized[i], sep="\t", file=f)

In [193]:
process_file()

KeyboardInterrupt: 